# 1. Konfiguracja środowiska
Aby przygotowany Notebook działał poprawnie trzeba wskazać katalogi, w których znajdują się dane.


*   __`reflex_project_path`__ - katalog głowny repozytorium, w którym umieszone są wszystkie wymagane biblioteki
*   __`raw_data_files`__ - folder, w którym przechowywane są dane jako surowe tablice `.npy.bz2`
* __`results_dir`__ - folder wynikowy, do którego zapiszemy wszystkie wygenerowane wyniki
* __`default_branch`__ - branch, z którego korzystamy.
* __`overwrite_files`__ - wartość w przypadku przeprowadzania eksprymentu wszystkie pliki zostają nadpisne nowymi (#TODO)
* __`experiment_name`__ - nazwa eksperymentu, ktory zostanie zapisany pod taka nazwe w folderze data.


In [0]:
#@title Konfiguracja zmiennych:

import sys
reflex_project_path = "/home/reflex/reflex" #@param {type:"string"}
default_branch = "master" #@param {type:"string"}
raw_data_files = "/mnt/disks/raw/all" #@param {type:"string"}
results_dir = "/home/reflex/reflex/data/results" #@param {type:"string"}
overwrite_files = True #@param ["True", "False"] {type:"raw"}
experiment_name = "labeler_768" #@param {type:"string"}

results_dir += f'_{experiment_name}' 
sys.path.append(reflex_project_path)

# 2. Aktualizacja lokalnego repozytorium
Zakładam, że repozytorium zostało już założone na maszynie wirtualnej.

In [34]:
!cd '{reflex_project_path}' && pwd && git checkout '{default_branch}' && git pull origin '{default_branch}'

/home/reflex/reflex
M	modules/radial_compressor.py
Already on 'master'
Your branch is up to date with 'origin/master'.
From gitlab.cs.put.poznan.pl:dbrzezinski/refleX
 * branch            master     -> FETCH_HEAD
Already up to date.


# 3. Instalowanie zależności
W przypadku uruchamiania skryptu na maszynach wirtualnych warto upewnić się, że wszystkie zależności zostały spełnine. Może być przydatne w procesie przenoszenia kodu na inne maszyny i może skrócić czas debugowania.

In [0]:
#!python3 -m pip install -r '{reflex_project_path}/requirements.txt'
#!conda install --file '{reflex_project_path}/requirements.txt'

# 3. Pełny preprocessing

## 3.1 Importowanie modułów


In [0]:
%load_ext autoreload
%autoreload 2

#Obsluga plikow .npy.bz2
from modules.rawdatalib import RawDataFile

# Wyznaczanie srodka oraz kata, pod ktorym pada promien
from modules.preprocessinglib import calculate_center, find_ray_angle

# Kompresja obrazu do wektora 1D
from modules.radial_compressor import main_external_call, make_ray_mask

# Funkcje pomocnicze
import modules.util as util

# Pozostale
import os
import cv2 as cv
import time
import bz2
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from multiprocessing import Pool, Lock

## 3.2 Tworzenie struktury katalogów

In [0]:
# Tworzenie folderu wynikowego
if not os.path.exists(results_dir):
  os.makedirs(results_dir)
  
# Tworzenie zmiennych pomocniczych
csv_dir = f'{results_dir}/csv'
images_dir = f'{results_dir}/images'
vectors_dir = f'{results_dir}/vectors'
normalized_npy_arrays_dir = f'{results_dir}/vectors'

# Tworzenie katalogow posrednich, w ktorych bedziemy trzymac dane
abs_dirs = [csv_dir, images_dir, vectors_dir, normalized_npy_arrays_dir]
  
# Tworzenie folderow posrednich
for dir in abs_dirs:
  if not os.path.exists(dir):
    os.makedirs(dir)

## 3.3 Etap I. - Wczytywanie, normalizowanie i zapisywanie jako obraz surowych danych.

In [4]:
def npy_to_img(idx, raw_file_path):
    
    # Okreslamy rozmiar obrazu, ktory ma zostac wygenerowany
    img_size = 768
  
    # Dla kazdego pliku generowany jest jego obiekt, ktory normalizuje obraz
    # oraz moze wyodrebniac informacje z metadanych (w przyszlosci mozemy
    # pozyskiwac srodek obrazu z zalaczonych do niego informacji w pliku .info)
    raw_file = RawDataFile(raw_file_path)
    image = raw_file.to_img(size=img_size)
    
    if 'beam_x' in raw_file.info.keys() and 'beam_y' in raw_file.info.keys() and 'distance' in raw_file.info.keys() and 'wavelength' in raw_file.info.keys():
      with lock:
        # WALIDOWAC CZY WPIS NIE ISTNIEJE!
        # TWORZYC PUSTY PLIK CSV ?
        util.write_to_csv(f'{csv_dir}/metadata_centers.csv', 
        {
           'image' : raw_file.filename,
           'x' : raw_file.info['beam_x'], 
           'y' : raw_file.info['beam_y'],
           'distance' : raw_file.info['distance'],
           'wavelength' : raw_file.info['wavelength']
        })
      
    cv.imwrite(f'{images_dir}/{raw_file.filename}.png', image)
    
    # Informacja
    print(f"Przetworzono: {raw_file.filename} | ({idx + 1}/{total})")
    
    #Nie img -> .npy

if os.path.exists(raw_data_files):
  
  # Pobranie listy plikow z folderu
  raw_files = [f'{raw_data_files}/{file}' for file in os.listdir(raw_data_files) if file.endswith('.npy.bz2')]
  
  # Liczba wszystkich plikow
  total = len(raw_files)
  
  # Rozpoczynami pomiar czasu
  start = time.time()

  # Lock
  lock = Lock()
    
  Parallel(n_jobs=8)(delayed(npy_to_img)(idx, raw_file_path) 
                     for idx, raw_file_path in enumerate(raw_files))    
    
  # Konczymy pomiar czasu
  end = time.time()
  time_str = time.strftime("%H:%M:%S", time.gmtime(end - start))
  
  print(f"\n --- KONIEC --- \n Przetworzono: {total} plikow, czas przetwarzania: {time_str}s")
  
else:
  raise FileNotFoundError("Katalog z plikami RAW nie istnieje!")

Przetworzono: 201817_1_00001 | (7/6129)
Przetworzono: khc3_4_1_001 | (8/6129)
Przetworzono: prv7-6 | (2/6129)
Przetworzono: idp05232b | (4/6129)
Przetworzono: 217343a5_cx#### | (12/6129)
Przetworzono: 90419_3_001 | (6/6129)
Przetworzono: 58118_1 | (3/6129)
Przetworzono: 46904_2_001 | (1/6129)
Przetworzono: 88089_1_E2_001 | (5/6129)
Przetworzono: 248067c1_x0001 | (16/6129)
Przetworzono: quk9-3_5_00001 | (11/6129)
Przetworzono: 30723_1_E1 | (9/6129)
Przetworzono: IDP91967-3b | (10/6129)
Przetworzono: xtal4-peg2-4c-x1.0001 | (14/6129)
Przetworzono: ny-znac-m3g6-2.0001 | (17/6129)
Przetworzono: 70568_1_E1_1000 | (13/6129)
Przetworzono: 40976_1_E2_001 | (22/6129)
Przetworzono: 96990_1_E1 | (15/6129)
Przetworzono: IDP02237-3b | (19/6129)
Przetworzono: 57037_1_E1_001 | (20/6129)
Przetworzono: 108276_1_001 | (18/6129)
Przetworzono: 229338F10_x0001 | (29/6129)
Przetworzono: 15521_2_001 | (25/6129)
Przetworzono: idp01616d | (21/6129)
Przetworzono: 48960_2_001 | (23/6129)
Przetworzono: 46035_1_E1

Przetworzono: sal9_p11_1_001 | (210/6129)
Przetworzono: 200500_1_00001 | (209/6129)
Przetworzono: 188360_1_E2_00001 | (215/6129)
Przetworzono: 103946_2_E1_001 | (211/6129)
Przetworzono: 139453_1_E1_0001 | (212/6129)
Przetworzono: 104883_1_001 | (213/6129)
Przetworzono: 21204_1_E1_001 | (216/6129)
Przetworzono: 188066_1_E1_001 | (217/6129)
Przetworzono: 97408_1_E1 | (214/6129)
Przetworzono: 15625_1_E1_001 | (218/6129)
Przetworzono: E4-drop2-xtal1-suc.0001 | (224/6129)
Przetworzono: set1_0001 | (219/6129)
Przetworzono: 79374_3_001 | (220/6129)
Przetworzono: 185801_3_E2_001 | (222/6129)
Przetworzono: IDP04364_4FO4.0001 | (223/6129)
Przetworzono: p8_c11_00001 | (227/6129)
Przetworzono: 255936e1_y0001 | (232/6129)
Przetworzono: 64991_3 | (221/6129)
Przetworzono: 67115_1_E2_001 | (225/6129)
Przetworzono: 57076_2_001 | (226/6129)
Przetworzono: 92948_1_001 | (228/6129)
Przetworzono: 185601_2_001 | (233/6129)
Przetworzono: 152414_1_E1_0001 | (229/6129)
Przetworzono: 153752_2_0001 | (231/6129)
P

Przetworzono: 93347_1_E2_001 | (412/6129)
Przetworzono: 194045_1_E1_00001 | (421/6129)
Przetworzono: 49967_1_E1_001 | (414/6129)
Przetworzono: 138148_2_0001 | (413/6129)
Przetworzono: 12058_1_E1_001 | (424/6129)
Przetworzono: 149178_1_E2_0001 | (418/6129)
Przetworzono: 137226_1_E2_0001 | (415/6129)
Przetworzono: 37581_1_E1_001 | (419/6129)
Przetworzono: aak7-1 | (423/6129)
Przetworzono: hok2-8 | (422/6129)
Przetworzono: infl | (430/6129)
Przetworzono: 112290_1_E2_001 | (425/6129)
Przetworzono: 112290_3_001 | (426/6129)
Przetworzono: 226144e11_x0001 | (434/6129)
Przetworzono: 131032_1_E3_001 | (427/6129)
Przetworzono: 165956_1_E1_0001 | (428/6129)
Przetworzono: 88778_1_E2_001 | (429/6129)
Przetworzono: ayk0-1_7_0001 | (433/6129)
Przetworzono: 14795_1_E2_001 | (431/6129)
Przetworzono: oaf3-9 | (436/6129)
Przetworzono: ia4a5g-peak.0001 | (435/6129)
Przetworzono: 218457h10_x0001 | (442/6129)
Przetworzono: 131780_3_E1_0001 | (432/6129)
Przetworzono: 20886_1_E1_001 | (438/6129)
Przetworzono:

Przetworzono: tlg6_2_1_092 | (621/6129)
Przetworzono: pos4_jb207r1.0001 | (622/6129)
Przetworzono: 17820_2 | (617/6129)
Przetworzono: 14851_2_001 | (626/6129)
Przetworzono: 151666_1_E1_0001 | (618/6129)
Przetworzono: sacol_1320-2pk_B | (624/6129)
Przetworzono: 103051_1_E1_001 | (625/6129)
Przetworzono: idp04423d | (627/6129)
Przetworzono: 161645_1_E1_001 | (631/6129)
Przetworzono: m1b4eg2.0001 | (630/6129)
Przetworzono: 36656_1_E1_001 | (628/6129)
Przetworzono: 105426_2_001 | (629/6129)
Przetworzono: 186724_1_001 | (633/6129)
Przetworzono: wdz1-9 | (632/6129)
Przetworzono: 195051_2_00001 | (639/6129)
Przetworzono: 225457h2_x0001 | (642/6129)
Przetworzono: idp90785a | (638/6129)
Przetworzono: 91113_2_001 | (634/6129)
Przetworzono: 142609_1_E1_0001 | (635/6129)
Przetworzono: T2972_1_E2_001 | (641/6129)
Przetworzono: 107516_1_E1_001 | (637/6129)
Przetworzono: 49157_2_001 | (636/6129)
Przetworzono: 228044h10_z0001 | (647/6129)
Przetworzono: 27025_2_001 | (640/6129)
Przetworzono: 65654_2_00

Przetworzono: 149539_1_E1_0001 | (824/6129)
Przetworzono: 59460_2_001 | (828/6129)
Przetworzono: 14775_3_001 | (823/6129)
Przetworzono: 153601_1_E1_0001 | (825/6129)
Przetworzono: 95613_1_E2 | (829/6129)
Przetworzono: 94284_2 | (830/6129)
Przetworzono: 84799_1_E1_001 | (831/6129)
Przetworzono: 264489b1_x0001 | (837/6129)
Przetworzono: 63766_2 | (832/6129)
Przetworzono: 11921_3_E2_001 | (839/6129)
Przetworzono: TM1782B4_4_E2_001 | (838/6129)
Przetworzono: 57548_2_001 | (834/6129)
Przetworzono: mj67_d10a | (835/6129)
Przetworzono: 197391_3_00001 | (841/6129)
Przetworzono: 167991_2_0001 | (833/6129)
Przetworzono: 143138_1_E2_0001 | (836/6129)
Przetworzono: 236774g7_x#### | (846/6129)
Przetworzono: IDP91546-1a | (843/6129)
Przetworzono: 152274_1_E2_0001 | (840/6129)
Przetworzono: sbq8-1 | (847/6129)
Przetworzono: 253960h9_hr_x0001 | (851/6129)
Przetworzono: 103946_2_E2_001 | (842/6129)
Przetworzono: CUX9-8_1_001 | (850/6129)
Przetworzono: 159700_2_0001 | (844/6129)
Przetworzono: 149178_2_0

...

Przetworzono: 173580_1_0001 | (5284/6129)
Przetworzono: met3_15_1_001 | (5290/6129)
Przetworzono: 70941_2_001 | (5288/6129)
Przetworzono: 91610_2_001 | (5289/6129)
Przetworzono: chy-pid2g-2.0001 | (5294/6129)
Przetworzono: STAH0001-1_B | (5296/6129)
Przetworzono: 13498_2_001 | (5299/6129)
Przetworzono: ozx5-4 | (5297/6129)
Przetworzono: 55315_1_E1_001 | (5292/6129)
Przetworzono: 166670_1_E2_0001 | (5291/6129)
Przetworzono: 111501_1_E2_001 | (5293/6129)
Przetworzono: native-ph-4.6-x0001 | (5303/6129)
Przetworzono: 36626_1_E1_001 | (5295/6129)
Przetworzono: dzb0-7_5_001 | (5301/6129)
Przetworzono: 60655_1_E2_001 | (5302/6129)
Przetworzono: IDP01765GMP_B.0001 | (5307/6129)
Przetworzono: 134075_1_E1_0001 | (5298/6129)
Przetworzono: EIL5-11_1_001 | (5306/6129)
Przetworzono: 149288_1_E1_0001 | (5300/6129)
Przetworzono: 93315_1_E1_001 | (5304/6129)
Przetworzono: 86977_2_E2_001 | (5305/6129)
Przetworzono: AQU7-6_1_001 | (5310/6129)
Przetworzono: 14867_3_001 | (5311/6129)
Przetworzono: 75674_1_

Przetworzono: 286982c9_x_0001 | (5498/6129)
Przetworzono: 97429_1_E1 | (5486/6129)
Przetworzono: 49046_1_E1_001 | (5488/6129)
Przetworzono: 56281_1_E2_001 | (5492/6129)
Przetworzono: qmk2-10 | (5494/6129)
Przetworzono: 233841f11_z0001 | (5503/6129)
Przetworzono: IDP00832_ct2-1_0001 | (5501/6129)
Przetworzono: 95613_2_E1 | (5495/6129)
Przetworzono: 90907_1_E1_001 | (5496/6129)
Przetworzono: 178893_1_E2_00001 | (5502/6129)
Przetworzono: 53256_1_E1_001 | (5497/6129)
Przetworzono: 159525_2_E2_0001 | (5500/6129)
Przetworzono: 179344_1_E2_00001 | (5506/6129)
Przetworzono: 139747_1_0001 | (5499/6129)
Przetworzono: 131780_1_E2_0001 | (5505/6129)
Przetworzono: CUX9-4_1_001 | (5510/6129)
Przetworzono: 12525_4_E2_001 | (5511/6129)
Przetworzono: 58046_2 | (5504/6129)
Przetworzono: 40464_1_E1_001 | (5507/6129)
Przetworzono: 148396_2_0001 | (5508/6129)
Przetworzono: idp00418_4PV4.0001 | (5515/6129)
Przetworzono: 41341_1_E2_001 | (5509/6129)
Przetworzono: 58118_2 | (5512/6129)
Przetworzono: idp00509_

Przetworzono: IDP01791_n1_2_0001 | (5694/6129)
Przetworzono: idp95081-c_1_1_8 | (5697/6129)
Przetworzono: 200827_2_00001 | (5700/6129)
Przetworzono: 200198_2_00001 | (5702/6129)
Przetworzono: ozx5-5_y | (5698/6129)
Przetworzono: STR0003-1d | (5699/6129)
Przetworzono: 90419_2_E2_001 | (5695/6129)
Przetworzono: 177823_1_00001 | (5705/6129)
Przetworzono: KWM-7-10_1_### | (5703/6129)
Przetworzono: 36658_2_001 | (5710/6129)
Przetworzono: kur4-6 | (5701/6129)
Przetworzono: dzb0-7_3_001 | (5709/6129)
Przetworzono: 99049_2_001 | (5704/6129)
Przetworzono: 127124_1_E2_001 | (5706/6129)
Przetworzono: 37842_2_001 | (5708/6129)
Przetworzono: 233128a2_x0001 | (5717/6129)
Przetworzono: 150864_1_E2_0001 | (5707/6129)
Przetworzono: xtal1_data_00001 | (5715/6129)
Przetworzono: 77848_1_E2_001 | (5714/6129)
Przetworzono: 271705c8_x_0001 | (5720/6129)
Przetworzono: D9_1 | (5711/6129)
Przetworzono: 264490e5_z20001 | (5722/6129)
Przetworzono: kyb3-5_3_001 | (5716/6129)
Przetworzono: idp02329-10 | (5712/6129)

Przetworzono: 159878_1_0001 | (5891/6129)
Przetworzono: 19408_1_E1_001 | (5903/6129)
Przetworzono: 20247_2_001 | (5906/6129)
Przetworzono: IUT-8-9_1_001 | (5902/6129)
Przetworzono: 169745_2_001 | (5900/6129)
Przetworzono: m1h2-peak.0001 | (5904/6129)
Przetworzono: vbd7-5 | (5905/6129)
Przetworzono: 13193_2_001 | (5910/6129)
Przetworzono: 157538_1_E2_0001 | (5901/6129)
Przetworzono: 197295_1_E2_00001 | (5912/6129)
Przetworzono: 54025_2_001 | (5907/6129)
Przetworzono: IDP02372_c200910n2_1_0001 | (5914/6129)
Przetworzono: blue5 | (5908/6129)
Przetworzono: 153223_2_0001 | (5909/6129)
Przetworzono: 178959_1_00001 | (5911/6129)
Przetworzono: 126486_2_001 | (5913/6129)
Przetworzono: pip-c9ge-1peak.0001 | (5920/6129)
Przetworzono: 169102_1_E2_0001 | (5916/6129)
Przetworzono: 156446_1_0001 | (5915/6129)
Przetworzono: 99202_1_E2_001 | (5917/6129)
Przetworzono: 167720_1_E1_0001 | (5918/6129)
Przetworzono: m1e5-1peak.0001 | (5922/6129)
Przetworzono: 83476_1_E2_001 | (5919/6129)
Przetworzono: 18099

Przetworzono: nsa3_16_1_001 | (6103/6129)
Przetworzono: 126867_1_001 | (6101/6129)
Przetworzono: 209126_2_00001 | (6102/6129)
Przetworzono: 185556_1_E2_001 | (6104/6129)
Przetworzono: IDP02065-5_2_A | (6105/6129)
Przetworzono: oth155-5 | (6109/6129)
Przetworzono: cp3a11g.0001 | (6108/6129)
Przetworzono: 99202_2_001 | (6106/6129)
Przetworzono: ood3-3_2_001 | (6111/6129)
Przetworzono: 169077_1_E2_0001 | (6107/6129)
Przetworzono: sMAT_N159G | (6115/6129)
Przetworzono: IDP00044_A4_C5_1_1_0001 | (6110/6129)
Przetworzono: 67156_1_E1_001 | (6112/6129)
Przetworzono: 18095_1_E1_001 | (6116/6129)
Przetworzono: 176568_1_E1_00001 | (6119/6129)
Przetworzono: 22317_1_E1_001 | (6113/6129)
Przetworzono: 120427_1_E1_001 | (6114/6129)
Przetworzono: wjm1-6 | (6118/6129)
Przetworzono: idp90278c | (6120/6129)
Przetworzono: CAA0034_collect_Dec13_2016_0001 | (6121/6129)
Przetworzono: 61021_1_E1_001 | (6122/6129)
Przetworzono: 91597_2_001 | (6117/6129)
Przetworzono: IDP5262ACo_B.0001 | (6126/6129)
Przetworzon

## 3.4 Etap II. - Okreslanie srodka obrazu oraz kata padania promienia.

In [0]:
def img_to_csv(idx, img_file_path):
  
    # Dla kazdego pliku wyznaczamy jego srodek oraz kat promienia
    filename = img_file_path[img_file_path.rfind('/') + 1:]
    image = cv.imread(img_file_path, cv.IMREAD_GRAYSCALE)
    _, _, _, center = calculate_center(image)
    ray_angle = find_ray_angle(image, center)
    
    if not ray_angle:
      ray_angle = (-1, -1)
      
    with lock:
      util.write_to_csv(f'{csv_dir}/centers.csv', 
                   {
                       'image' : filename,
                       'x' : center[0], 
                       'y' : center[1],
                       'mask_start' : ray_angle[0],
                       'mask_end' : ray_angle[1]
                   })

    # Informacja
    print(f"Przetworzono: {filename} | ({idx + 1}/{total})")
    
if os.path.exists(csv_dir):
  
  # Pobranie listy obrazow
  img_files = [f'{images_dir}/{file}' for file in os.listdir(images_dir) if file.endswith('.png')]

  # Liczba wszystkich plikow
  total = len(raw_files)
  
  # Rozpoczynami pomiar czasu
  start = time.time()
  
  # Lock
  lock = Lock()
  
  
  # TODO: Return !Results
  Parallel(n_jobs=8)(delayed(img_to_csv)(idx, img_file_path) 
                   for idx, img_file_path in enumerate(img_files))  
    
  # Konczymy pomiar czasu
  end = time.time()
  time_str = time.strftime("%H:%M:%S", time.gmtime(end - start))
  
  print(f"\n --- KONIEC --- \n Przetworzono: {total} plikow, czas przetwarzania: {time_str}s")
  
else:
  raise FileNotFoundError("Katalog CSV nie istnieje!")

### Wizualizacja wyników etapu II.

In [1]:
%matplotlib inline    


def visualize(img, center_x, center_y, mask_start=-1, mask_end=-1, center_color=(0, 0, 0)):
    center_x = int(center_x)
    center_y = int(center_y)

    radius = int(min(center_x, center_y, img.shape[0]-center_y, img.shape[1]-center_x))
    img = cv.cvtColor(img, cv.COLOR_GRAY2RGB)
    cv.circle(img, (center_x, center_y), 3, center_color, thickness=-1)
    cv.circle(img, (center_x, center_y), int(radius/2), center_color, thickness=1)
    cv.circle(img, (center_x, center_y), radius, center_color, thickness=1)

    ray_mask = make_ray_mask(img, (int(center_x), int(center_y)), mask_start, mask_end)
    return cv.addWeighted(ray_mask, 0.2, img, 0.8, 0)

      
def get_przemek_dict(json_path):
  import json
  with open(json_path) as json_data:
    d = json.load(json_data)
    return d
    

def numpy_shape_from_filename(filename):
  data = np.load(bz2.BZ2File(filename))
  return data.shape
    

def visualize_main():
  import matplotlib.pyplot as plt
  num_examples = 5
  
  df = pd.read_csv(f'{csv_dir}/centers.csv')
  metadata = pd.read_csv(f'{csv_dir}/metadata_centers.csv')
  df['image'] =  df['image'].map(lambda x: x[:-12])
  df.set_index('image', inplace=True)
  metadata.set_index('image', inplace=True)
  df = df.join(metadata, lsuffix='_ours', rsuffix='_przemka', how='inner')
  df = df.drop_duplicates()
  przemkowe = get_przemek_dict(f'{reflex_project_path}/przemkowe/pixel_sizes.json')

  i = 0
  for index, row in df.iterrows():
    if i >= num_examples:
      break
    i = i + 1  
    
    im_name_core = row.name
    img_filename = f'{images_dir}/{im_name_core}.512x512.png'
    np_filename = f'{raw_data_files}/{im_name_core}.npy.bz2'
    if not os.path.isfile(img_filename):
      raise FileNotFoundError("Invalid image filename: " + img_filename)
    if not os.path.isfile(np_filename):
      raise FileNotFoundError("Invalid numpy filename: " + np_filename)
    
    img = cv.imread(img_filename, cv.IMREAD_GRAYSCALE)
    
    our_img = visualize(img,
              row['x_ours'],
              row['y_ours'],
              row['mask_start'],
              row['mask_end'],
              center_color=(0,255,0)
             )
    
    width_coeff = przemkowe[im_name_core]['width']
    height_coeff = przemkowe[im_name_core]['height']
    metadata_x = row['x_przemka']
    metadata_y = row['y_przemka']
    wavelength = row['wavelength']
    distance = row['distance']
    np_shape = numpy_shape_from_filename(np_filename)
    
    converted_x = img.shape[1] * metadata_x / width_coeff / np_shape[1]
    converted_y = img.shape[0] * metadata_y / height_coeff / np_shape[0]
    
    #print(converted_x, converted_y)
    przemek_img = visualize(img,
              converted_x,
              converted_y,
              center_color=(255,0,0)
             )
    _, ax = plt.subplots(figsize=(16, 8))
    ax.imshow(np.hstack((our_img, przemek_img)))

visualize_main()

NameError: name 'pd' is not defined

## 3.5 Etap III. - Kompresowanie obrazu do wektora 1D

In [0]:
if os.path.exists(vectors_dir):
  
  # Plik ze srodkami
  centers_file = f'{csv_dir}/centers.csv'
  
  # Rozpoczynami pomiar czasu
  start = time.time()

  # Kompresja do wektorow z uzyciem Parallel
  main_external_call(images_dir, vectors_dir + '/', f'{csv_dir}/centers.csv', 8, 240)
    
  # Konczymy pomiar czasu
  end = time.time()
  time_str = time.strftime("%H:%M:%S", time.gmtime(end - start))
  
  print(f"\n --- KONIEC --- \n Przetworzono: {total} plikow, czas przetwarzania: {time_str} s")
  
else:
  raise FileNotFoundError("Katalog CSV nie istnieje!")